## File for different Model Layers and the like

In [1]:
import torch
from torch import nn

In [2]:
s = torch.rand(8,20,20)
v = torch.rand(8,20,1)

torch.matmul(s,v).shape

torch.Size([8, 20, 1])

In [3]:
class selfAttentionDot(nn.Module):
    """calculating dot product self attention
    """
    def __init__(self):
        super(selfAttentionDot,self).__init__()
    
    def forward(self,q,k,v):
        #q,k,v = (batch_shape, n)
        # add activations
        depth = q.shape[1]
        k = torch.unsqueeze(k,dim = 2) #(b,n,1)
        q = torch.unsqueeze(q,dim = 1) #(b,1,n)
        scores = torch.matmul(q,k)/torch.sqrt(depth) #(b,n,n)
        v = torch.unsqueeze(v, dim = 2)
        values = scores*v #(b,n)
        values = torch.squeeze(dim = 2)
        
        return values
        
        
        

In [32]:
test = torch.tensor([[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
print(test,test.shape)


tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]]) torch.Size([4, 3])


In [37]:
# test = torch.tensor([[1,1,1],[2,2,2],[3,3,3],[4,4,4]]).float()
# mean = torch.mean(test, dim = 1,keepdim=True)
# print(mean)
# # print(test.shape,mean.shape)
# mean = mean.repeat(1,3)
# # print(test.shape,mean.shape)
# # torch.sub(test,mean,)
# print(test)
# print(mean)
# print(test-mean)

In [38]:
class LayerNorm1D(nn.Module):
    """My implementation of LayerNorm for 1d inputs from https://arxiv.org/pdf/1607.06450.pdf
    still have to figure out how this links up with a (batch_dim,channel,seq_length) input for transformers
    as well as fix some shapes
    """
    def __init__(self, input_dim):
        super(LayerNorm1D,self).__init__()
        self.input_dim = input_dim
        self.bias = nn.parameter.Parameter(data=torch.zeros(1,input_dim),requires_grad=True)
        self.gain = nn.parameter.Parameter(data=torch.ones(1,input_dim),requires_grad=True)
    
    def forward(self,inputs):
        """
        Args:
            inputs (torch.Tensor): tensor of shape(B, input_dim)
        """
        batch_dim = inputs.shape[0]
        mu = torch.mean(inputs,dim=1)
        var = torch.mean(torch.square(inputs - mu.repeat(1,self.input_dim)),dim = 1)
        std = torch.sqrt(var)
        
        centered = (inputs - mu.repeat(1,self.inpu_dim))
        
        gain = self.gain.repeat(batch_dim, 1)
        bias = self.bias.repeat(batch_dim, 1)
        
        scale = std/gain
        
        norm = scale*centered + bias
        return norm
        
        
        

In [39]:
l = LayerNorm1D(4)

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self,size):
        super(TransformerEncoder,self).__init__()
        
        self.keys = nn.Linear(size, size)
        self.query = nn.Linear(size, size)
        self.values = nn.Linear(size, size)
        
        self.selfAttention = selfAttentionDot()
        
    def forward(self,inputs):
        k = self.keys(inputs)
        q = self.query(inputs)
        v = self.values(inputs)
        attention = self.selfAttention(k,q,v)
        res = attention + inputs
                
        
        
        